In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import linalg as la
from matplotlib import animation
plt.switch_backend('QT5AGG')

# Problem 1

In [ ]:
a = 0.
b = 1.
t_init = 0.
t_final = 1.

x_int = 6
t_int = 10

h = (b-a)/x_int
k = (t_final-t_init)/t_int

v = .05
lamda = v*k/h**2

U = np.zeros((t_int+1,x_int+1))

x_val = np.linspace(a,b,x_int+1)

def f(x):
    return 2*np.maximum(.2-np.abs(x-.5),0)

U[0,:] = f(x_val)

A = np.zeros((x_int+1,x_int+1))
A[0,0] = 1
A[-1,-1] = 1
A[1:-1,1:-1] = np.diag((1-2*lamda)*np.ones(x_int-1)) + np.diag(lamda*np.ones(x_int-2),-1) + np.diag(lamda*np.ones(x_int-2),1)
A[1,0] = lamda
A[-2,-1] = lamda

for j in range(1,int(t_int)+1):
    U[j] = A.dot(U[j-1])

plt.plot(x_val,U[4])
plt.show()

# Problem 2

In [ ]:
a = -12.
b = 12.
t_0 = 0.
t_f = 1.

t_int = 70
x_int = 140

h = (b-a)/x_int
k = (t_f-t_0)/t_int

v = 1.
lamda = v*k/h**2

U = np.zeros((t_int+1,x_int+1))

x_val = np.linspace(a,b,x_int+1)

def f(x):
    return np.maximum(1-x**2,0)

U[0,:] = f(x_val)

A = np.zeros((x_int+1,x_int+1))
A[0,0] = 1
A[-1,-1] = 1
A[1:-1,1:-1] = np.diag((1-2*lamda)*np.ones(x_int-1)) + np.diag(lamda*np.ones(x_int-2),-1) + np.diag(lamda*np.ones(x_int-2),1)
A[1,0] = lamda
A[-2,-1] = lamda

for j in range(1,int(t_int)+1):
    U[j] = A.dot(U[j-1])

def heat_flow_animation():
    f = plt.figure()
    plt.axes(xlim=(-12,12),ylim=(-.1,1))
    line, = plt.plot([],[])
    def animate(i):
        line.set_data(x_val, U[i])
        return line,
    
    a = animation.FuncAnimation(f,animate,frames=U.shape[0],interval=50)
    plt.show()
    
heat_flow_animation()

In [ ]:
#This is the bad animation, when lambda is too big

a = -12.
b = 12.
t_0 = 0.
t_f = 1.

t_int = 66
x_int = 140

h = (b-a)/x_int
k = (t_f-t_0)/t_int

v = 1.
lamda = v*k/h**2

U = np.zeros((t_int+1,x_int+1))

x_val = np.linspace(a,b,x_int+1)

def f(x):
    return np.maximum(1-x**2,0)

U[0,:] = f(x_val)

A = np.zeros((x_int+1,x_int+1))
A[0,0] = 1
A[-1,-1] = 1
A[1:-1,1:-1] = np.diag((1-2*lamda)*np.ones(x_int-1)) + np.diag(lamda*np.ones(x_int-2),-1) + np.diag(lamda*np.ones(x_int-2),1)
A[1,0] = lamda
A[-2,-1] = lamda

for j in range(1,int(t_int)+1):
    U[j] = A.dot(U[j-1])

def heat_flow_animation():
    f = plt.figure()
    plt.axes(xlim=(-12,12),ylim=(-.1,1))
    line, = plt.plot([],[])
    def animate(i):
        line.set_data(x_val, U[i])
        return line,
    
    a = animation.FuncAnimation(f,animate,frames=U.shape[0],interval=50)
    plt.show()
    
heat_flow_animation()

# Problem 3

In [ ]:
def Crank_Nicolson(a,b,t_0,t_f,x_int,t_int,nu,f):
    h = float(b-a)/x_int
    k = float(t_f-t_0)/t_int
    
    lamda = nu*k/(2*h**2)
    
    U = np.zeros((t_int+1,x_int+1))
    
    x_val = np.linspace(a,b,x_int+1)
    
    U[0] = f(x_val)
    
    A = np.zeros((x_int+1,x_int+1))
    A[0,0] = 1
    A[-1,-1] = 1
    A[1:-1,1:-1] = np.diag((1-2*lamda)*np.ones(x_int-1)) + np.diag(lamda*np.ones(x_int-2),-1) + np.diag(lamda*np.ones(x_int-2),1)
    A[1,0] = lamda
    A[-2,-1] = lamda
    
    B = np.zeros_like(A)
    B[0,0] = 1
    B[-1,-1] = 1
    B[1:-1,1:-1] = np.diag((1+2*lamda)*np.ones(x_int-1)) - np.diag(lamda*np.ones(x_int-2),-1) - np.diag(lamda*np.ones(x_int-2),1)
    B[1,0] = -lamda
    B[-2,-1] = -lamda
    
    B_inv = la.inv(B)
    
    for j in range(1,int(t_int)+1):
        U[j] = B_inv.dot(A.dot(U[j-1]))
    
    return U

def f(x):
    return np.maximum(1-x**2,0)

a = -12.
b = 12.
t_0 = 0.
t_f = 1.

nu = 1.

U_true = Crank_Nicolson(a,b,t_0,t_f,640,640,nu,f)[-1,::32]

U_list = []
error = []
h_list = []

for i in range(5):
    steps = 20*(2**(i))
    h_list.append(float(b-a)/steps)
    total = int(steps/20)
    U = Crank_Nicolson(a,b,t_0,t_f,steps,steps,nu,f)[-1,::total]
    error.append(np.max(np.abs(U-U_true)))

h_list = np.array(h_list)

plt.loglog(h_list,error,'.-k',label='Error')
plt.loglog(h_list,h_list**2,label='$h^2$')
plt.legend(loc=0)
plt.show()